In [258]:
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.actions.action_builder import ActionBuilder
import pyautogui as pag
import time
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin


In [259]:
os.environ['PATH'] += r"D:/Anaconda/anaconda3"
driver = webdriver.Firefox()
driver.maximize_window()

In [260]:
driver.get('https://permits.cityofboise.org/CitizenAccess/Welcome.aspx')



In [261]:
link = driver.find_element(By.XPATH, '/html/body/form/div[3]/header/div/nav/div/div/ul/li[2]/span/div')
link.click()

WebDriverWait(driver,30).until(
    EC.text_to_be_present_in_element(
        (By.XPATH,'/html/body/form/div[3]/header/div/nav/div/div/ul/li[2]/div/a[5]'),
        'Licenses'
    )
)

lk2 = driver.find_element(By.XPATH, '//*[@id="cobnav"]/div/ul/li[2]/div/a[2]') 

lk2.click()



In [262]:
rec_type = driver.find_element(By.ID, 'ctl00_PlaceHolderMain_generalSearchForm_ddlGSPermitType')



rec_type.click()

sel_rec = driver.find_element(By.XPATH, '/html/body/form/div[4]/div/div[7]/div[1]/table/tbody/tr/td/div[2]/div[3]/div/div/div[2]/div[1]/div/div[1]/div[1]/div[1]/div[1]/div[2]/table/tbody/tr/td/div/div[2]/table/tbody/tr/td[1]/select/option[2]')

sel_rec.click()

# #Calendar
# button = driver.find_element(By.ID,'ctl00_PlaceHolderMain_generalSearchForm_txtGSStartDate_calendar_button')

# driver.execute_script("document.getElementById('ctl00_PlaceHolderMain_generalSearchForm_txtGSStartDate_calendar_button').click()")
# driver.execute_script("document.getElementById('ctl00_PlaceHolderMain_generalSearchForm_txtGSStartDate_calendar_bhv_lnkTitleID').click()")

## Calendar

In [263]:
scroll_origin = ScrollOrigin.from_viewport(10, 10)
ActionChains(driver)\
.scroll_from_origin(scroll_origin, 0, 800)\
.perform()

In [264]:
time.sleep(4)

In [265]:

pag.moveTo(480,250)
pag.click()
time.sleep(1)
pag.moveTo(440,280)
pag.click()
time.sleep(1)
pag.moveTo(390,350)
pag.click()
time.sleep(1)
pag.moveTo(430,340)
pag.click()
time.sleep(1)

In [266]:
find_eng = driver.find_element(By.ID, 'ctl00_PlaceHolderMain_btnNewSearch')
driver.execute_script("document.getElementById('ctl00_PlaceHolderMain_btnNewSearch').click()")

## Search

In [274]:
scroll_origin = ScrollOrigin.from_viewport(10, 10)
ActionChains(driver)\
.scroll_from_origin(scroll_origin, 0, -10000)\
.perform()

## Permits scraping

In [275]:
driver.execute_script("document.getElementById('ctl00_PlaceHolderMain_dgvPermitList_gdvPermitList_ctl02_hlPermitNumber').click()")

In [ ]:
permit_num = 
project_name = 
status = 
project_st = 
project_city = 
licenced_professional = 
professional_city = 
professional_zip = 
applicant_name = 
applicant_company = 
applicant_main_phone = 
applicant_email = 